In [1]:
import pandas as pd
import numpy as np
import json
import os
from scipy.special import softmax
from tqdm.notebook import tqdm

In [2]:
def process_dicts(folders, result_dict_name="default_dict"):
    # Создаем выходные папки
    probs_folder = os.path.join(os.getcwd(), result_dict_name + "_probs")
    max_folder = os.path.join(os.getcwd(), result_dict_name + "_max")
    os.makedirs(probs_folder, exist_ok=True)
    os.makedirs(max_folder, exist_ok=True)

    for folder in folders:
        for file in tqdm(os.listdir(folder)):
            if not file.endswith(".json"):
                continue

            file_path = os.path.join(folder, file)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # Получаем исходные данные
            predictions = data["valid_predictions"]
            architecture = data["architecture"]
            accuracy = data["valid_accuracy"]

            # Softmax преобразование
            softmax_preds = softmax(predictions, axis=1).tolist()
            softmax_output = {
                "architecture": architecture,
                "test_predictions": softmax_preds,
                "test_accuracy": accuracy,
            }
            with open(os.path.join(probs_folder, file), "w", encoding="utf-8") as f_out:
                json.dump(softmax_output, f_out)

            # Argmax преобразование
            argmax_preds = np.argmax(predictions, axis=1).tolist()
            argmax_output = {
                "architecture": architecture,
                "test_predictions": argmax_preds,
                "test_accuracy": accuracy,
            }
            with open(os.path.join(max_folder, file), "w", encoding="utf-8") as f_out:
                json.dump(argmax_output, f_out)

In [3]:
process_dicts(["cifar100"], "cifar100")

  0%|          | 0/51 [00:00<?, ?it/s]